In [5]:
## Load Imports ##
import nibabel as nib
import numpy as np

import os
import sysX
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import dice_coef
from utils import computeAverageDice

0 in validation set.


/home/shared/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/shared/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
import matplotlib.pyplot as plt
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
      axes[i].imshow(slice.T, cmap="gray", origin="lower")

def show_slice(slice):
    plt.imshow(slice.T, cmap="gray", origin="lower")
    
def normalizeImg(x):
    # Normalize x
    max_val = np.max(x)
    min_val = np.min(x)
    norm_x = (x-min_val)/(max_val-min_val + 1e-7)
    return norm_x

In [12]:
# Search for best dice score

# Range of thresh to iterate over
log_range = np.arange(-0.5, 0, 0.01)
thresh_range = 10**log_range
# Ground truth and predicted directory
gt_dir = '../slice_data_side/'
pred_dir = '../slice_data_side_pred/'

dice_scores = []
for thresh in thresh_range:
    dc = computeAverageDice(thresh, pred_dir, gt_dir)
    print(dc)
    dice_scores.append(dc)

print(dice_scores)
    

63770 in validation set.
0.8391147540912843
63770 in validation set.
0.8412108664980665
63770 in validation set.
0.8433001945295082
63770 in validation set.
0.8452469433959109
63770 in validation set.
0.8470764896924353
63770 in validation set.
0.8488691258364847
63770 in validation set.
0.8505009829200103
63770 in validation set.
0.8521245848714802
63770 in validation set.
0.853689584597902
63770 in validation set.
0.8551071682459431
63770 in validation set.
0.8564518610254898
63770 in validation set.
0.8577136015623974
63770 in validation set.
0.8588729314087157
63770 in validation set.
0.8600217415377847
63770 in validation set.
0.8611224640737906
63770 in validation set.
0.8621393396135656
63770 in validation set.
0.8630386416736573
63770 in validation set.
0.8638681298820838
63770 in validation set.
0.8646595023920692
63770 in validation set.
0.8653288737258266
63770 in validation set.
0.8659120576061197
63770 in validation set.
0.866367806078599
63770 in validation set.
0.8667454

In [14]:
idx = np.argmax(dice_scores)
print(thresh_range[idx])

0.5623413251903494


In [16]:
print(computeAverageDice(thresh_range[idx], pred_dir, gt_dir))

63770 in validation set.
0.8671948656340756


In [5]:
## Prepare list of tuples of paths to GT & Predicted Masks ##
gt_dir = '../slice_data_side/'
pred_dir = '../slice_data_side_pred/'
mask_tups = []

for _,_,files in os.walk(gt_dir):
    for file in files:
        # Prepare GT & pred fp
        if 'mask' not in file:
            continue
        pred_file = file.replace('side_mask_', 'side_mask_pred_')
        gt_fp = gt_dir + file
        pred_fp = pred_dir + pred_file
        # Create list of tuples (gt, pred)
        mask_tups.append((gt_fp, pred_fp))


        
        

In [30]:
## Compute Dice Scores across alls scans ##
import random
# Randomly sample from mask_tups
sampled_masks = random.sample(mask_tups, len(mask_tups)//22)
print(len(sampled_masks))
dice_coefs = []
for idx, mask_tup in enumerate(sampled_masks):
    #if (not os.path.isfile(mask_tup[0]) or not os.path.isfile(mask_tup[1])):
    #    print(mask_tup[0], 'or', mask_tup[1], 'doesn\'t exist?')
    #    continue
    #print(mask_tup[1])
    gt_mask = (nib.load(mask_tup[0])).get_data()
    pred_mask = (nib.load(mask_tup[1])).get_data()
    dice_coefs.append(dice_coef(gt_mask, pred_mask, threshold = 0.5))
    if idx%1000 == 0:
        print(idx)
    
    

3577
0
1000


/home/julien/ARFF/metrics.py:13: RuntimeWarning: invalid value encountered in double_scalars
  (np.sum(inv_mask_true) + np.sum(inv_mask_pred))))


2000
3000


In [37]:
from scipy import stats
dc = np.array(dice_coefs)
dc = dc[~np.isnan(dc)]
print(stats.describe(dc))

# Get best max
bm_idx = np.argmax(dc)
gt_mask = (sampled_masks[bm_idx])[0]
pred_mask = (sampled_masks[bm_idx])[1]
gt_mask_img = nib.load(gt_mask).get_data()



DescribeResult(nobs=3576, minmax=(0.0, 0.989607597204802), mean=0.8603695924711291, variance=0.021903653829475768, skewness=-3.244207536033756, kurtosis=11.838710183373417)
